In [1]:
import cv2
from find_image import FindImage

In [2]:
find_image = FindImage(False)

In [3]:
img = find_image.find('6da4c7ee12790d8047e82c7750884780')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'cascade.xml')
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=1, minSize=(100,100))

In [5]:
known_cat = find_image.find('50fcbd042ae9a24e3afd5cbb8e1e4542')
known_cat_gray = cv2.cvtColor(known_cat, cv2.COLOR_BGR2GRAY)

In [6]:
known_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'cascade.xml')
known_face_rects = face_cascade.detectMultiScale(known_cat_gray, scaleFactor=1.1, minNeighbors=1)

In [7]:
for (x, y, w, h) in known_face_rects:
    known_face_roi = known_cat_gray[y:y+h, x:x+w]
    known_face_roi = cv2.resize(known_face_roi, (150, 150))
    break
else:
    raise Exception("Não foi possível encontrar o rosto que se deseja reconhecer")

In [8]:
for (x, y, w, h) in faces:
    face_roi = gray[y:y+h, x:x+w]
    face_roi = cv2.resize(face_roi, (150, 150))

    result = cv2.matchTemplate(known_face_roi, face_roi, cv2.TM_CCOEFF_NORMED)
    print(result)
    if result[0][0] > 0.7:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)


cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[-0.04290152]]
[[0.00079856]]
[[-0.01386347]]
[[0.12339736]]
